# Launch your digital twin: Edge harvester
This is a KWH application (notebook) which can be used to build and launch the digital twin whose configurtion file was created with the notebook 08d. The launched digital twin will automatically connect to S³I and it can be reached using S³I-B messages. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I and modelling language library.

In [ ]:
import s3i
import time 
import os 
from ml.tools import load_config
from ml.dt_factory import create_dt_ref, build_feature, add_function_impl_obj
from ml.fml40.features.functionalities.accepts_felling_jobs import AcceptsFellingJobs
from ml.ml40.features.properties.values.documents.jobs.job_status import JobStatus
from ml.app_logger import APP_LOGGER, setup_logger
from tools import print_with_timestamp, yes, no, hmi_id, hmi_secret

## Load the configuration file 

In [ ]:
dt_name = input("[S³I]: Please enter the name of your digital twin (my_edge_harvester): ")
config_file_name = "{}.json".format(dt_name)
root_path = os.path.abspath("")
dt_model = load_config(config_file_name, root=root_path)


## Set up a logger

In [ ]:
setup_logger(dt_model["attributes"].get("name", None))

## Build the digital twin 
The construction of the digital twin is based on the configuration file created previously. Features, roles and another relevant entries, which are composed in the configuration file, will be instantiated with the resptive object of the fml40 library. 

In [ ]:
dt_secret = input("[S³I]: Please enter the secret of your digital twin: ")
dt = create_dt_ref(model=dt_model, grant_type="client_credentials", 
                   secret=dt_secret,
                   is_broker_rest=True,
                   is_broker=True, is_repo=False)

To show the extensibility of fml40, we have implemented a class that inherits all the methods and properties of the fml40::AcceptsFellingJobs class. Additionally, we have also pre-defined a simulate function, which simulates the rpm changing of harvester's engine. 

In [ ]:
class AcceptsFellingJobsImpl(AcceptsFellingJobs):
    """
    This class inherits the method acceptJob, queryJobStatus and removejob of the fml40::AcceptsFellingJobs class. 
    """
    def __init__(self, name="", identifier=""):
        super(AcceptsFellingJobs, self).__init__(
            name=name,
            identifier=identifier)
        self.job_list = []

    def acceptJob(self, job):
        APP_LOGGER.info("Checking if the felling job can be accepted.")
        if isinstance(job, dict):
            try:
                felling_job = build_feature(feature=job)
                for job in self.job_list:
                    if job.identifier == felling_job.identifier:
                        APP_LOGGER.info("Job with ID {} has been rejected, because this job was already accepted".format(
                            felling_job.identifier))
                        return False

                felling_job.status = JobStatus.InProgress.name
                self.job_list.append(felling_job)
                APP_LOGGER.info("Job with ID {} has been accepted".format(felling_job.identifier))
                return True

            except:
                APP_LOGGER.info("Job with ID {} has been rejected".format(felling_job.identifier))
                return False

    def queryJobStatus(self, identifier):
        APP_LOGGER.info("Checking the job status of job {}".format(identifier))
        for job in self.job_list:
            if job.identifier == identifier:
                APP_LOGGER.info("Job {} is now in status {}".format(identifier, job.status))
                return {"identifier": identifier, "status": job.status}
        APP_LOGGER.info("Job {} can not be queried".format(identifier))
        return {"identifier": identifier, "status": "NOT FOUND"}

    def removeJob(self, identifier):
        APP_LOGGER.info("Checking if i can remove the job {}".format(identifier))
        for job in self.job_list:
            if job.identifier == identifier:
                self.job_list.remove(job)
                APP_LOGGER.info("Job {} removed".format(identifier))
                return True
        APP_LOGGER.info("Job {} can not be found".format(identifier))
        return False


def simulate_rpm():
    """
    This function simulates how the rpm value of the engine changes between 2000 to 2500 U/min. 
    """
    my_engine = dt.features["ml40::Composite"].targets["my_engine"]
    tank = "up"

    while True:
        if tank == "down":
            __new_rpm = my_engine.features["ml40::RotationalSpeed"].rpm - 10
            if __new_rpm < 2000:
                tank = "up"

        elif tank == "up":
            __new_rpm = my_engine.features["ml40::RotationalSpeed"].rpm + 10
            if __new_rpm > 2500:
                tank = "down"
        my_engine.features["ml40::RotationalSpeed"].rpm = __new_rpm
        time.sleep(1)

Insert the class and function into the instance of the digital twin. 

In [ ]:
add_function_impl_obj(dt, AcceptsFellingJobsImpl, "fml40::AcceptsFellingJobs")
dt.add_user_def(func=simulate_rpm)

## Launch the digital twin
The digital twin is then started and automatically connected to S³I . It will firstly authenticate itself at the S³I IdentityProvider.  Afterwards, a listener for the S³I Broker is also launched to receive the S³I-B messages. The digital twin has a thing entry at the S³I Directory and its original model is exclusively stored locally, so it is called as edge harvester.  

In [ ]:
dt.run_forever()
while True:
    i = input("[S³I]: End the digital twin? (j/n)")
    if i in yes:
        break
    elif i in no:
        continue